In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import torch
from skimage import measure
from tqdm import tqdm, trange

from core.dataset import ScanNet
from core.integrate import ScalableTSDFVolume
from core.integrate.utils import discrete2hash, discrete2world, discretize_3d, hash2discrete, floor_3d

In [3]:
# test if voxel_size = 0.01 and margin = 0.36 works, sdf_trunc = 0.04

In [4]:
dataset = ScanNet('/home/quanta/Datasets/ScanNet/')
scan_id = 'scene0000_00'
id = dataset.scan_id_list.index(scan_id)
single_instance = dataset[id]['scan_dataset']

In [5]:
tsdf_device = 'cuda:1'
tsdf_volume = ScalableTSDFVolume(
    voxel_size=0.01,
    sdf_trunc=0.04,
    margin=0.36,
    device=tsdf_device,
)

In [6]:
indent = 1
for idx in trange(0, len(single_instance), indent):
# for idx in trange(0, 500, indent):
    inputs = single_instance.get_torch_tensor(idx, device=tsdf_device, keys={"depth", "depth_intr", "pose"})
    tsdf_volume.integrate_tsdf(
        depth=inputs["depth"],
        depth_intr=inputs["depth_intr"],
        cam_pose=inputs["pose"],
    )

100%|██████████| 5578/5578 [08:10<00:00, 11.37it/s]


In [8]:
# 2.343G mem use
print(tsdf_volume.num_voxel)

73236092


In [9]:
tsdf_volume.prune_voxel()
torch.cuda.empty_cache()
# drops to 1.879 G mem use

In [11]:
print(tsdf_volume.num_voxel)

4838648


In [18]:
a = torch.zeros(
    size=(4838648, 512), dtype=torch.float32, device='cuda:1'
) #18.965g

In [19]:
torch.cuda.empty_cache()

In [21]:
4838648 * 512 * 4 / 1024 / 1024 / 1024

9.228988647460938